In [7]:
import git
import networkx as nx
import matplotlib.pyplot as plt

def show_graph(graph):
    plt.figure(figsize=(5, 60))

    pos = nx.nx_pydot.pydot_layout(graph, prog='dot')
    labels = {node: node[:7] for node in graph.nodes}
    nodeColors = [
        "red" if len(graph[node]) > 1 else "lightblue" 
        for node in graph.nodes
    ]

    nx.draw(graph, pos, labels=labels, font_size=8, node_color= nodeColors)


    plt.title("Commit Graph (Top to Bottom)")
    plt.show()


In [13]:
def save_graph(file_path, graph):
    with open(file_path, 'wb') as f:
        nx.write_edgelist(graph, f, data=False)

def process_repository(repo_path, output_path):
    repo = git.Repo(repo_path)
    
    g = {}

    for commit in repo.iter_commits('--all'):
        commit_hash = str(commit)
        parents = [str(parent) for parent in commit.parents]

        if commit_hash not in g:
            g[commit_hash] = set()
        g[commit_hash].update(parents)

    graph = nx.DiGraph(g)


    print(graph)
    #show_graph(graph)

    save_graph(output_path, graph)


# https://github.com/Sycnex/Windows10Debloater/commits/master/?before=a48b4d8dc501680e0edc31f840791c966d89d309+35
process_repository("C:/Users/Gerome/Desktop/graphes-git/dossiers_git/Windows10Debloater", "C:/Users/Gerome/Desktop/graphes-git/dossiers_git/test")

DiGraph with 353 nodes and 400 edges


In [ ]:
def load_graph(file_path):
    graph = nx.read_edgelist(file_path, create_using=nx.DiGraph())
    return graph

G = load_graph("C:/Users/Gerome/Desktop/graphes-git/dossiers_git/test")
#show_graph(G)

In [2]:
import json

def load_repos_from_json(filename):
    with open(filename, "r") as f:
        return json.load(f)

repos = load_repos_from_json("C:/Users/Gerome/Desktop/graphes-git/dossiers_git/repos_info.json")
print(repos)

[{'id': 33043794, 'language': 'Crystal', 'full_name': 'generate-cr/library.template', 'clone_url': 'https://github.com/generate-cr/library.template.git', 'forks_count': 1, 'open_issues_count': 1, 'size': 236, 'stargazers_count': 5, 'created_at': '2015-03-28T17:51:53Z', 'updated_at': 'generate-cr/library.template', 'pushed_at': '2015-04-11T21:56:53Z'}, {'id': 33077982, 'language': 'JavaScript', 'full_name': 'zpratt/coffee-hunt', 'clone_url': 'https://github.com/zpratt/coffee-hunt.git', 'forks_count': 1, 'open_issues_count': 0, 'size': 152, 'stargazers_count': 5, 'created_at': '2015-03-29T15:23:18Z', 'updated_at': 'zpratt/coffee-hunt', 'pushed_at': '2015-04-05T03:43:50Z'}, {'id': 33120021, 'language': 'JavaScript', 'full_name': 'vadimdemedes/leeroy-jenkins', 'clone_url': 'https://github.com/vadimdemedes/leeroy-jenkins.git', 'forks_count': 1, 'open_issues_count': 0, 'size': 685, 'stargazers_count': 5, 'created_at': '2015-03-30T11:46:47Z', 'updated_at': 'vadimdemedes/leeroy-jenkins', 'push